In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import pandas as pd
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df_train = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
df_test =  pd.read_csv("/kaggle/input/restaurant-revenue-prediction/test.csv.zip")

In [ ]:
df_train = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
#to display first 5 rows
df_train.head(5)

In [ ]:
#display basic structure of train data
df_train

In [ ]:
#Total no of rows and columns
df_train.shape

In [ ]:
#Displaying Column names
df_train.columns

In [ ]:
df_train.info

In [ ]:
#Checking the data type
df_train.dtypes

In [ ]:
#Structure of testing data
df_test.shape

In [ ]:
#Rows containing duplicate data
duplicate_rows_df_train = df_train[df_train.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df_train.shape)

In [ ]:
#count no of rows 
df_train.count()

In [ ]:
df_train_columns = [col for col in df_train.columns if col not in ["Id", "Open Date"]]


In [ ]:
df_train_columns

In [ ]:
#numerical columns
num_col=df_train.select_dtypes(exclude=['object']).columns
num_col

In [ ]:
#categorical columns
cat_col=df_train.select_dtypes(include=['object']).columns
cat_col

In [ ]:
#getting null values if any
df_train.columns[df_train.isna().any()].tolist()

In [ ]:
cityPerc=df_train[["City Group","revenue"]].groupby(['City Group'],as_index=False).mean()
sns.barplot(x='City Group', y='revenue',data=cityPerc)

In [ ]:
cityPerc=df_train[["City","revenue"]].groupby(['City'],as_index=False).mean()
newDF=cityPerc.sort_values(["revenue"],ascending=False)
sns.barplot(x='City', y='revenue',data=newDF.head(6))

In [ ]:
cityPerc=df_train[["City","revenue"]].groupby(['City'],as_index=False).mean()
newDF=cityPerc.sort_values(["revenue"],ascending=True)
sns.barplot(x='City', y='revenue',data=newDF.head(6))

In [ ]:
cityPerc=df_train[["Type","revenue"]].groupby(['Type'],as_index=False).mean()
sns.barplot(x='Type', y='revenue',data=cityPerc)

In [ ]:
df_train.head()

In [ ]:
correlation = df_train[num_col].corr()
f, ax=plt.subplots(figsize=(14,12))
plt.title('Correlation of numerical attributes',size=16)
sns.heatmap(correlation)
plt.show()

In [ ]:
correlation['revenue'].sort_values(ascending=False)

In [ ]:
#df_train=df_train.drop('Open Date', axis=1)
#df_train=df_train.drop('Id',axis=1)
#df_train.head(5)

In [ ]:
CityGroupDummyTrain=pd.get_dummies(df_train['City Group'])
df_train=df_train.join(CityGroupDummyTrain)
#df_train=df_train.drop('City Group', axis=1)
df_train.head()

In [ ]:
# Data type conversion
df_train = df_train.astype({"City":'category',"City Group":'category',"Type":'category', "Open Date":'datetime64[ns]'})
# df_train.info()
df_test = df_test.astype({"City":'category',"City Group":'category',"Type":'category', "Open Date":'datetime64[ns]'})
# df_test.info()

In [ ]:
# 1. Check categories in train and test data
# df_train : City, City group, Type
cities = df_train.City.unique().tolist()
print(len(cities))
city_group = df_train['City Group'].unique().tolist()
print(city_group)
type_ = df_train['Type'].unique().tolist()
print(type_)

In [ ]:
cities = df_test.City.unique().tolist()
print(len(cities))
city_group = df_train['City Group'].unique().tolist()
print(city_group)
type_ = df_train['Type'].unique().tolist()
print(type_)

In [ ]:
df_train['City Group'] = df_train[['City Group']].apply(lambda x: x.cat.codes)
df_train['Type'] = df_train[['Type']].apply(lambda x: x.cat.codes)
from sklearn.tree import DecisionTreeRegressor

df_train = df_train.drop(columns = ['Id','Open Date','City'],axis=1)
X = df_train.loc[:, df_train.columns != 'revenue']
y = df_train.loc[:, df_train.columns == 'revenue']

tree = DecisionTreeRegressor().fit(X, y)
# Feature importance dataframe
d = {'Importance': np.round(tree.feature_importances_, 3) , 'Features': X.columns}
feature_imp = pd.DataFrame(data=d)
# Sort
feature_imp_order = feature_imp.sort_values('Importance', ascending=False).reset_index(drop=True)[0:20]
feature_imp_order

In [ ]:
# Training
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


X_main = df_train[feature_imp_order.Features.values]
y_main = df_train['revenue']
scaler_train = StandardScaler()
scaler_train.fit(X_main)
X_main_scaled = scaler_train.transform(X_main)

# Splitting the data into training and testing data
linear_regr = LinearRegression()
linear_regr.fit(X_main_scaled, y)
y_pred = linear_regr.predict(X_main_scaled)
accuracy = linear_regr.score(X_main_scaled,y)
print("Train Accuracy {}%".format(int(round(accuracy *100))))
print("Training RMSE Linear regression ",mean_squared_error(y, y_pred, squared=False))

from sklearn.svm import SVR
svm_regr = SVR(kernel = 'rbf')
svm_regr.fit(X_main_scaled, y)
y_pred = svm_regr.predict(X_main_scaled)
accuracy = svm_regr.score(X_main_scaled,y)
print("Train Accuracy {}%".format(int(round(accuracy *100))))
print("Training RMSE SVM regressor ",mean_squared_error(y, y_pred, squared=False))

# import the regressor
from sklearn.tree import DecisionTreeRegressor 
  
# create a regressor object
dt_regr = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
dt_regr.fit(X_main_scaled, y)
y_pred = dt_regr.predict(X_main_scaled)
accuracy = dt_regr.score(X_main_scaled,y)
print("Train Accuracy {}%".format(int(round(accuracy *100))))
print("Training RMSE SVM regressor ",mean_squared_error(y, y_pred, squared=False))

In [ ]:
# Prepare test data
df_test
df_test['City Group'] = df_test[['City Group']].apply(lambda x: x.cat.codes)
df_test['Type'] = df_test[['Type']].apply(lambda x: x.cat.codes)

result_df_linear_regr = pd.DataFrame()
result_df_linear_regr['Id'] = df_test['Id'].values

result_df_svm_regr = pd.DataFrame()
result_df_svm_regr['Id'] = df_test['Id'].values

result_df_dt_regr = pd.DataFrame()
result_df_dt_regr['Id'] = df_test['Id'].values

df_test = df_test[feature_imp_order.Features.values]

X_test = df_test.loc[:, df_test.columns != 'revenue']
# y_test = df_test.loc[:, df_train.columns == 'revenue']

scaler_test = StandardScaler()
scaler_test.fit(X_test)
X_test_scaled = scaler_test.transform(X_test)
y_pred_linear_regr = linear_regr.predict(X_test_scaled)
result_df_linear_regr['Prediction'] = y_pred_linear_regr
result_df_linear_regr['Prediction'] = result_df_linear_regr['Prediction'].apply(lambda x:round(x,2))
result_df_linear_regr

y_pred_svm_regr = svm_regr.predict(X_test_scaled)
result_df_svm_regr['Prediction'] = y_pred_svm_regr
result_df_svm_regr['Prediction'] = result_df_svm_regr['Prediction'].apply(lambda x:round(x,2))
result_df_svm_regr

y_pred_dt_regr = dt_regr.predict(X_test_scaled)
result_df_dt_regr['Prediction'] = y_pred_dt_regr
result_df_dt_regr['Prediction'] = result_df_dt_regr['Prediction'].apply(lambda x:round(x,2))
result_df_dt_regr.to_csv('Prediction_DT.csv',index=False)
result_df_dt_regr